In [1]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf

from src.dataset import *
from src.model import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
# TFRWriter().write()

In [ ]:
config = {
    'local_depth': 16,
    'local_convdepth': 32,
    'global_depth': 5,
    'global_convdepth': 64,
    'global_imagesize': 128,
    'scale': 0.25}

model = CIEXYZNet(**config)

In [ ]:
val_data = DataLoader().get_dataset('validation')

In [ ]:
for i, val_batch in enumerate(val_data):
    logits = model(val_batch['sRGB_image'])
    print(i, logits)

In [ ]:
model.summary()

In [ ]:
model.srgb2xyz_localnet.summary()

In [ ]:
model.srgb2xyz_globalnet.summary()